In [84]:
import numpy as np
import pandas as pd
import os
import pysam
import matplotlib.pyplot as plt
import seaborn as sns
import pybedtools
from collections import defaultdict
import itertools
import subprocess
import re

In [71]:
# prep for downstream analysis
os.mkdir("analysis")
os.mkdir("results/diff_region")
data_save = "results/diff_region/"
analysis_save = "analysis/diff_region/"

ATAC-seq Differential Binding Analysis Steps
To identify differential binding regions in ATAC-seq data when you already have peak calls and deduplicated, filtered, sorted BAM files, here are the main steps involved:
1. Count reads in peaks for each sample

Create a consensus peak set by merging peaks from all samples
Count reads that fall within these peaks for each sample
Generate a count matrix (peaks × samples)

2. Normalize the count data

Account for sequencing depth differences
Apply appropriate normalization methods (TMM, RLE, etc.)

3. Perform differential analysis

Use statistical frameworks like DESeq2, edgeR, or limma
Model the data considering experimental design and covariates
Test for significant differences between conditions

4. Filter and annotate results

Apply significance thresholds (p-value, FDR)
Filter by fold change magnitude
Annotate regions with nearby genes

5. Visualization and downstream analysis

Create heatmaps, MA plots, or PCA plots
Perform motif enrichment analysis in differential regions
Pathway analysis of genes near differential regions

# Step1 Count reads in peaks
 - Peak Union Method
 - Fisher's Method with Genrich (Not shown due to outdated)
 - PeakMerge Approach
 - Pseudobulk Peak Calling (Not shown)
 - regions (promoters and enhancers; independent of peak calling)

In [58]:
# Peak Union Method
## get overlapping regions for any combination of two peak sets
## get union of all overlapping regions
## filter peaks based on chr1-22 & X
## count reads in peaks using bam files

peak_files = [os.path.join('results/peaks',i) for i in os.listdir('results/peaks')
              if i.endswith(".narrowPeak")]
peak_files = [pybedtools.BedTool(i) for i in peak_files]
print(peak_files)

def three_columns(peaks):
    peaks = '\n'.join([f"{entry.chrom}\t{entry.start}\t{entry.end}" for entry in peaks])
    peaks = pybedtools.BedTool(peaks, from_string=True)
    return peaks

peak_files = [three_columns(i) for i in peak_files]

Ctrl_peak_files = peak_files[:3]
TrT_peak_files = peak_files[3:6]

[<BedTool(results/peaks/GSF4007-Control_1_S11_peaks.narrowPeak)>, <BedTool(results/peaks/GSF4007-Control_2_S13_peaks.narrowPeak)>, <BedTool(results/peaks/GSF4007-Control_3_S15_peaks.narrowPeak)>, <BedTool(results/peaks/GSF4007-NICD3-V5_1_S12_peaks.narrowPeak)>, <BedTool(results/peaks/GSF4007-NICD3-V5_2_S14_peaks.narrowPeak)>, <BedTool(results/peaks/GSF4007-NICD3-V5_3_S16_peaks.narrowPeak)>]


In [78]:
def union_overlapping_peaks(bed_a, bed_b, percentage=50):
    """Find overlapping regions with overlap percentage information."""
    # Get overlaps with the original features and overlap base pairs
    overlaps = bed_a.intersect(bed_b, wo=True)
    
    # Process results to include percentage information
    result_data = []
    for overlap in overlaps:
        chrom = overlap[0]
        start_overlap = int(overlap[1])  # Start of feature A
        end_overlap = int(overlap[2])    # End of feature A
        
        # These indices may need adjustment based on your BED format
        start_b = int(overlap[4])  # Start of feature B
        end_b = int(overlap[5])    # End of feature B
        
        # The intersection is the actual overlapping region
        intersect_start = max(start_overlap, start_b)
        intersect_end = min(end_overlap, end_b)
        
        # Calculate overlap size and percentage
        overlap_bp = int(overlap[-1])
        length_a = end_overlap - start_overlap
        percent_of_a = (overlap_bp / length_a) * 100
        length_b = end_b - start_b
        percent_of_b = (overlap_bp / length_b) * 100
        
        # Store the overlapping region with metadata
        if percent_of_a >= percentage and percent_of_b >= percentage:
            result_data.append([
                chrom, 
                intersect_start, 
                intersect_end
            ])
    
    # Convert to DataFrame for easier handling
    if result_data:
        df = pd.DataFrame(
            result_data, 
            columns=["chrom", "start", "end"]
        )
        return pybedtools.BedTool.from_dataframe(df)
    else:
        return pybedtools.BedTool.from_dataframe(pd.DataFrame(columns=["chrom", "start", "end"]))


def combined_peaksets(peak_files):
    combined = pybedtools.BedTool.from_dataframe(pd.DataFrame(columns=["chrom", "start", "end"]))
    for i,j in itertools.combinations(range(3),2):
        combined = combined.cat(union_overlapping_peaks(peak_files[i], peak_files[j]), postmerge=False)

    combined = combined.sort().merge()
    combined = combined.filter(lambda x: x.chrom in ["chr"+str(i) for i in range(1,23)]+["chrX"])
    return combined

Ctrl_peaks = combined_peaksets(Ctrl_peak_files)
TrT_peaks = combined_peaksets(TrT_peak_files)

Ctrl_peaks = Ctrl_peaks.saveas()
TrT_peaks = TrT_peaks.saveas()
Merged_peaks = Ctrl_peaks.cat(TrT_peaks, postmerge=False)
Merged_peaks = Merged_peaks.sort().merge()
Merged_peaks = Merged_peaks.to_dataframe()

Merged_peaks.to_csv(data_save+'PeakUnion_peaks.bed', sep="\t", index=False, header=False)

In [ ]:
def count_reads_in_peaks(bam_file, peak_file, paired_end=True):
    """
    Count reads in peaks from BAM file
    
    Args:
        bam_file: Path to BAM file
        peak_file: Path to BED file with peaks
        paired_end: Whether BAM file contains paired-end reads
        
    Returns:
        DataFrame with peak coordinates and read counts
    """
    # Open BAM file
    bam = pysam.AlignmentFile(bam_file, "rb")
    
    # Read peaks
    peaks = []
    with open(peak_file, 'r') as f:
        for line in f:
            fields = line.strip().split('\t')
            chrom, start, end = fields[0], int(fields[1]), int(fields[2])
            peaks.append((chrom, start, end))
    
    # Count reads in each peak
    counts = []
    for chrom, start, end in peaks:
        if paired_end:
            # For paired-end, count fragments (not individual reads)
            read_count = sum(1 for read in bam.fetch(chrom, start, end) 
                            if read.is_proper_pair and read.is_read1)  # Count only read1 to avoid counting fragments twice
        else:
            # For single-end
            read_count = sum(1 for read in bam.fetch(chrom, start, end))
        
        counts.append(read_count)
    
    # Create result DataFrame
    result = pd.DataFrame({
        'chrom': [p[0] for p in peaks],
        'start': [p[1] for p in peaks],
        'end': [p[2] for p in peaks],
        'count': counts
    })
    
    return result

bam_files = [os.path.join("results/blacklist_filtered",i) for i in os.listdir('results/blacklist_filtered')
             if i.endswith('nobl.bam')]

combined = pd.DataFrame(columns=['chrom','start','end','count','sample'])
for i in bam_files:
    res = count_reads_in_peaks(bam_file=i, peak_file=data_save+'PeakUnion_peaks.bed', 
                     paired_end=True)
    res['sample'] = re.sub('.nobl.bam','',i)
    combined = pd.concat([combined, res], axis=0,ignore_index=True)


In [81]:
%%bash
awk 'OFS="\t" {print $1"."$2"."$3,$1,$2,$3,"."}' results/diff_region/PeakUnion_peaks.bed > out.saf
featureCounts -T 20 --countReadPairs -p -a out.saf -F SAF -o results/diff_region/PeakUnion_counts.txt results/blacklist_filtered/*.nobl.bam



        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.1.1

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 6 BAM files                                      ||
||                                                                            ||
||                           GSF4007-Control_1_S11.nobl.bam                   ||
||                           GSF4007-Control_2_S13.nobl.bam                   ||
||                           GSF4007-Control_3_S15.nobl.bam      

In [88]:
# Peak Merge Method
## get union of all regions
## filter peaks based on chr1-22 & X
## count reads in peaks using bam files


peak_files = [os.path.join('results/peaks',i) for i in os.listdir('results/peaks')
              if i.endswith(".narrowPeak")]

with open('All_peaks.bed', 'w') as outfile:
    for peak_file in peak_files:
        with open(peak_file, 'r') as infile:
            outfile.write(infile.read())

combined = pybedtools.BedTool("All_peaks.bed")
combined = combined.filter(lambda x: x.chrom in ["chr"+str(i) for i in range(1,23)]+["chrX"])

combined = combined.saveas()
combined = combined.to_dataframe()
combined.to_csv(data_save+'PeakMerge_peaks.bed', sep="\t", index=False, header=False)






In [90]:
%%bash
awk 'OFS="\t" {print $1"."$2"."$3,$1,$2,$3,"."}' results/diff_region/PeakMerge_peaks.bed > out.saf
featureCounts -T 20 --countReadPairs -p -a out.saf -F SAF -o results/diff_region/PeakMerge_counts.txt results/blacklist_filtered/*.nobl.bam



        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.1.1

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 6 BAM files                                      ||
||                                                                            ||
||                           GSF4007-Control_1_S11.nobl.bam                   ||
||                           GSF4007-Control_2_S13.nobl.bam                   ||
||                           GSF4007-Control_3_S15.nobl.bam      

In [ ]:
# regions
## download promoter and enhancer regions from UCSC
## get upstream/downstream 3/5kb of TSS of genes as promoter regions

df_promoters = pd.read_table("ref/promoter.bed", sep="\t", header=None)
df_promoters = df_promoters[df_promoters[0].isin(["chr"+str(i) for i in range(1,23)]+["chrX"])]
df_enhancers = pd.read_table("ref/enhancer.bed", sep="\t", header=None)
df_enhancers = df_enhancers[df_enhancers[0].isin(["chr"+str(i) for i in range(1,23)]+["chrX"])]


df_enhancers.to_csv('ref/'+'enhancer_chr1-22X.bed', sep="\t", index=False, header=False)
df_promoters.to_csv('ref/'+'promoter_chr1-22X.bed', sep="\t", index=False, header=False)


In [107]:
%%bash
awk 'OFS="\t" {print $1"."$2"."$3,$1,$2,$3,"."}' ref/promoter_chr1-22X.bed > out.saf
featureCounts -T 20 --countReadPairs -p -a out.saf -F SAF -o results/diff_region/promoter_chr1-22X.txt results/blacklist_filtered/*.nobl.bam

awk 'OFS="\t" {print $1"."$2"."$3,$1,$2,$3,"."}' ref/enhancer_chr1-22X.bed > out.saf
featureCounts -T 20 --countReadPairs -p -a out.saf -F SAF -o results/diff_region/enhancer_chr1-22X.txt results/blacklist_filtered/*.nobl.bam

awk 'OFS="\t" {print $1"."$2"."$3,$2,$3,$4,"."}' ref/Promoter_UCSC_hg38_3000bp_chr1-22X.bed > out.saf
featureCounts -T 20 --countReadPairs -p -a out.saf -F SAF -o results/diff_region/Promoter_UCSC_hg38_3000bp_chr1-22X.txt results/blacklist_filtered/*.nobl.bam

awk 'OFS="\t" {print $1"."$2"."$3,$2,$3,$4,"."}' ref/Promoter_UCSC_hg38_5000bp_chr1-22X.bed > out.saf
featureCounts -T 20 --countReadPairs -p -a out.saf -F SAF -o results/diff_region/Promoter_UCSC_hg38_5000bp_chr1-22X.txt results/blacklist_filtered/*.nobl.bam



        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.1.1

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 6 BAM files                                      ||
||                                                                            ||
||                           GSF4007-Control_1_S11.nobl.bam                   ||
||                           GSF4007-Control_2_S13.nobl.bam                   ||
||                           GSF4007-Control_3_S15.nobl.bam      